# ECG analysis project
### Author: Marian Petruk


### Import libraries, modules

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import warnings
import pandas as pd
from scipy.signal import butter, sosfilt, sosfilt_zi, sosfiltfilt, lfilter, lfilter_zi, filtfilt, sosfreqz, resample
from utils import hamilton_detector, christov_detector, findpeaks, engzee_detector
from ecg_detectors.ecgdetectors import Detectors, MWA, panPeakDetect, searchBack
import os
import csv
from numpy import genfromtxt
import pyhrv.tools as tools
from biosppy import utils
import statistics 



### Load raw ECG signal

In [2]:


#SAMPLE_FILE='sample2-external';
SAMPLE_FILE='sample1-default';

temp=pd.read_csv("samples/"+str(SAMPLE_FILE)+".csv")

data=np.empty([len(temp)])
for x in range(len(temp)):
    data[x]=temp.values[x][0]


### Plot raw ecg sample

### Filter the data from noice

#### **Butterworth Bandpass filter**

In [3]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog=False, btype="band", output="sos")
    return sos


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfilt(sos,
                data)  # Filter data along one dimension using cascaded second-order sections. Using lfilter for each second-order section.
    return y


def butter_bandpass_filter_once(data, lowcut, highcut, fs, order=5):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    # Apply the filter to data. Use lfilter_zi to choose the initial condition of the filter.
    zi = sosfilt_zi(sos)
    z, _ = sosfilt(sos, data, zi=zi * data[0])
    return sos, z, zi


def butter_bandpass_filter_again(sos, z, zi):
    # Apply the filter again, to have a result filtered at an order the same as filtfilt.
    z2, _ = sosfilt(sos, z, zi=zi * z[0])
    return z2


def butter_bandpass_forward_backward_filter(data, lowcut, highcut, fs, order=5):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfiltfilt(sos,
                    data)  # Apply a digital filter forward and backward to a signal.This function applies a linear digital filter twice, once forward and once backwards. The combined filter has zero phase and a filter order twice that of the original.
    return y

As we can see, it is better to use `scipy.signal.sosfiltfilt` instead of `sosfilt` to apply the Butterworth filter. `sosfiltfilt` is the forward-backward filter. It applies the filter twice, once forward and once backward, resulting in zero phase delay.

#### Illustration of a phase delay/shift

Phase shift is any change that occurs in the phase of one quantity, or in the phase difference between two or more quantities. (c) Wikipedia 
<img width="300px" src="images/Phase_shift.png" alt="illustration of phase shift from wikipedia">

We can see phase delay on filtered singal using `sosfilt` without initial conditions and on both stages of `sosfilt` with initial conditions (`sosfilt_zi`). However, `sosfiltfilt` - a forward-backward digital filter has **zero phase delay/shift**.

## QRS

QRS detection is difficult, not only because of the physiological variability of the QRS complexes, but also because of the various types of noise that can be present in the ECG signal. Noise sources include muscle noise, artifacts due to electrode motion, power-line interference, baseline wander and T waves with high frequency characteristics similar to QRS complexes.

### Detect R-peaks

#### Main detector that is used is pan-tompkins

In [4]:
def pan_tompkins_detector(raw_ecg, mwa, fs, N):
   
#     N = int(0.12 * fs)
#     mwa = MWA(squared, N)
#     mwa[:int(0.2 * fs)] = 0

    N = int(N / 100 * fs)
    mwa_peaks = panPeakDetect(mwa, fs)

    r_peaks = searchBack(mwa_peaks, raw_ecg, N)

    return r_peaks


In [5]:
# Derivative - provides QRS slope information.
differentiated_ecg_measurements = np.ediff1d(y)

# Squaring - intensifies values received in derivative. 
# This helps restrict false positives caused by T waves with higher than usual spectral energies..
squared_ecg_measurements = differentiated_ecg_measurements ** 2

# Moving-window integration.
integration_window = 50  # Change proportionally when adjusting frequency (in samples)
integrated_ecg_measurements = np.convolve(squared_ecg_measurements, np.ones(integration_window))

# Fiducial mark - peak detection on integrated measurements.
rpeaks = pan_tompkins_detector(data, integrated_ecg_measurements, fs, integration_window)
print(rpeaks)



NameError: name 'y' is not defined

### RR-intervals (aka NN-intervals)

The term **`NN`** is used in place of **RR** to emphasize the fact that the processed beats are "normal" beats. (с) Wikipedia

Heart rate variability (HRV) is the physiological phenomenon of variation in the time interval between heartbeats. It is measured by the variation in the beat-to-beat interval.

In [ ]:
rr = np.diff(rpeaks) / fs * 1000  # in miliseconds
hr = 60 * 1000 / rr
print("rr =", rr)
print("hr =", hr)
rr

### HRV features
#### Time-domain methods

In [ ]:

def timeDomain(NN):
    
    L = len(NN)    
    ANN = np.mean(NN)
    SDNN = np.std(NN)
    SDSD = np.std(np.diff(NN))    
    NN50 = len(np.where(np.diff(NN) > 0.05)[0])    
    pNN50 = NN50/L    
    NN20 = len(np.where(np.diff(NN) > 0.02)[0])
    pNN20 = NN20/L
    rMSSD = np.sqrt((1/L) * sum(np.diff(NN) ** 2))        
    MedianNN = np.median(NN)
    SDANN = sdann(rr)['sdann']
    SDNN_INDEX=sdnn_index(rr)['sdnn_index']
    RRI_MEAN=RRI_Mean(rr)['rri_mean']
    
    timeDomainFeats = {
        "ANN": ANN, "SDNN": SDNN,
        "SDSD": SDSD, "NN50": NN50,
                       "SDANN":SDANN , 
                        "SDNN_INDEX": SDNN_INDEX,
                       "RRI_Mean": RRI_MEAN,
                       "pNN50": pNN50, "NN20": NN20,
                       "pNN20": pNN20, "rMSSD": rMSSD,
                       "MedianNN":MedianNN}
                       
    return timeDomainFeats


def sdnn_index(nni=None, rpeaks=None, full=True, overlap=False, duration=300):

    # Check input
    nn = tools.check_input(nni, rpeaks)

    # Signal segmentation into 5 min segments
    segments, seg = tools.segmentation(nn,  full=full, overlap=overlap, duration=duration)

    if seg:
        sdnn_values = [sdnn(x)['sdnn'] for x in segments]
        sdnn_index = np.mean(sdnn_values)
    else:
        sdnn_index = float('nan')
        if tools.WARN:
            warnings.warn("Signal duration too short for SDNN index computation.")

    # Output
    args = [sdnn_index]
    names = ['sdnn_index']
    return utils.ReturnTuple(args, names)


def sdann(nni=None, rpeaks=None, full=True, overlap=False, duration=300):

    # Check input
    nn = tools.check_input(nni, rpeaks)

    # Signal segmentation into 5 min segments
    segments, seg = tools.segmentation(nn, full=full, overlap=overlap, duration=duration)

    if seg:
        mean_values = [np.mean(x) for x in segments]
        sdann_ = tools.std(mean_values)
    else:
        sdann_ = float('nan')
        if tools.WARN:
            warnings.warn("Signal duration too short for SDANN computation.")

    # Output
    args = [sdann_]
    names = ['sdann']
    return utils.ReturnTuple(args, names)


def RRI_Mean(nni=None, rpeaks=None, full=True, overlap=False, duration=300):

    # Check input
    nn = tools.check_input(nni, rpeaks)
    
    # Signal segmentation into 5 min segments
    segments, seg = tools.segmentation(nn, full=full, overlap=overlap, duration=duration)
    if seg:
        rri_mean_ = statistics.mean([np.mean(x) for x in segments])

    else:
        rri_mean_ = float('nan')
        if tools.WARN:
            warnings.warn("Signal duration too short for RRI Mean computation.")

    # Output
    args = [rri_mean_]
    names = ['rri_mean']
    return utils.ReturnTuple(args, names)

In [ ]:

result = timeDomain(rr)
timeDomain(rr)

resultJSON=(str(result).replace("'",'"')).replace('nan','"nan"')

f = open("samples/"+SAMPLE_FILE+"-result.json", "w")
print(resultJSON)
f.write(resultJSON)
f.close()